In [ ]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

In [35]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

Loading data 

In [36]:
fsleyesDisplay.load(op.expandvars('T1w.nii'))

In [37]:
sample_path = "/data"
mkdir_no_exist(sample_path)
deriv_= op.join(sample_path, 'derivatives')

In [38]:
preproc_root = op.join(sample_path, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)

skull stripping with BET

In [39]:
def skull_stripped(sample_root, preproc_root, robust=False):
    """
    Perform skull stripping on a T1w anatomical image and save directly in preproc_root.
    
    Args:
        sample_root (str): Path to the directory containing the input T1w image.
        preproc_root (str): Path to save the skull-stripped outputs.
        robust (bool): Whether to use robust mode (-R) in BET.
    """
    anatomical_path = op.join(sample_root, 'T1w.nii.gz')
    betted_brain_path = op.join(preproc_root, 'T1w_stripped')
    os.system(f'bet {anatomical_path} {betted_brain_path} -m {"-R" if robust else ""}')
    print("Done with BET.")

In [40]:
skull_stripped(sample_path, preproc_root)

Done with BET.


In [41]:
resulting_mask_path = op.join(preproc_root, 'T1w_stripped_mask.nii.gz')

In [42]:
fsleyesDisplay.load(resulting_mask_path)

Setting Robust = TRUE 

In [43]:
skull_stripped(sample_path, preproc_root, robust=True)

Done with BET.


In [44]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(sample_path, 'T1w.nii'))
fsleyesDisplay.load(resulting_mask_path)


(ipykernel_launcher.py:406): Gdk-WARNING **: 15:16:36.700: gdkdrawable-x11.c:952 drawable is not a pixmap or window


Tissue segmentation 

In [49]:
anatomical_path = op.join(sample_path, 'T1w.nii.gz')
bet_path = op.join(preproc_root,'T1w_stripped.nii.gz') 

In [50]:
fast_target = bet_path 
[os.remove(f) for f in glob.glob(op.join(preproc_root, '*fast*'))] 
segmentation_path = op.join(preproc_root, 'T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

{}

In [51]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_root, '*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, '*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_root, '*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'